<a href="https://colab.research.google.com/github/ilia1221/test/blob/main/part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re, requests
import json
import torch

In [ ]:
url = 'https://raw.githubusercontent.com/phygitalism/test-tasks-3dml/master/Task1/dev.json'
resp = requests.get(url)
data = json.loads(resp.text)

In [ ]:
data[:2]

[{'dpdg1': 0.9571806007967657,
  'dpdg2': 0.2429918798353707,
  'id': 1,
  'x1': 0.5784676164550582,
  'x2': 0.25269502829221124},
 {'dpdg1': 1.5548069825999542,
  'dpdg2': 0.3202737235651227,
  'id': 2,
  'x1': 0.18004037024122943,
  'x2': 0.8573727538702526}]

In [ ]:
result = []
for point in data:
    dpdg1 = point['dpdg1']
    dpdg2 = point['dpdg2']

    x1 = torch.tensor([point['x1']], dtype=torch.float64, requires_grad=True)
    x2 = torch.tensor([point['x2']], dtype=torch.float64, requires_grad=True)

    f1 = x1 + x2
    f2 = x1 * x2
    g1 = torch.tan(f1 + f2 + 100)
    g2 = f1 * f2


    g1.backward(retain_graph=True)

    dg1_dx1 = x1.grad
    dg1_dx2 = x2.grad

    x1.grad.data.zero_()
    x2.grad.data.zero_()

    g2.backward(retain_graph=True)

    dg2_dx1 = x1.grad
    dg2_dx2 = x2.grad

    dydx1 = dpdg1 * dg1_dx1 + dpdg2 * dg2_dx1
    dydx2 = dpdg1 * dg1_dx2 + dpdg2 * dg2_dx2

    result.append({
         'id': point['id'],
         'dydx1': dydx1.item(),
         'dydx2': dydx2.item(),
        })



In [ ]:
import pandas as pd
df = pd.DataFrame(result)
df.to_csv('result.csv', index=False, header=False)